This Kernel mainly focuses on solving missing data.

In [1]:
import numpy as np
import pandas as pd

import datetime
import math

First we double check where are our input data is and what's the exact directory name. Then load the csv file as a dataframe.

In [2]:
!ls ../input

weather-dataset-rattle-package


In [3]:
rain = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


Now if we're focusing on solving this problem as a classification problem, we must drop the 'RISK_MM' as it leaks out the information in predicting our target variable 'RainTomorrow'.

In [4]:
#drop risk_mm
rain.drop(['RISK_MM'], axis=1, inplace=True)
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Let's look at the missing value stats. Computed by number of missing values in the column divided by total number of rows.

In [5]:
for col in rain.columns:
    print(col + ' has ' + str(round((rain[col].isnull().sum() / rain.shape[0]) * 100, 2)) + '% missing values')

Date has 0.0% missing values
Location has 0.0% missing values
MinTemp has 0.45% missing values
MaxTemp has 0.23% missing values
Rainfall has 0.99% missing values
Evaporation has 42.79% missing values
Sunshine has 47.69% missing values
WindGustDir has 6.56% missing values
WindGustSpeed has 6.52% missing values
WindDir9am has 7.04% missing values
WindDir3pm has 2.66% missing values
WindSpeed9am has 0.95% missing values
WindSpeed3pm has 1.85% missing values
Humidity9am has 1.25% missing values
Humidity3pm has 2.54% missing values
Pressure9am has 9.86% missing values
Pressure3pm has 9.83% missing values
Cloud9am has 37.74% missing values
Cloud3pm has 40.15% missing values
Temp9am has 0.64% missing values
Temp3pm has 1.92% missing values
RainToday has 0.99% missing values
RainTomorrow has 0.0% missing values


Then check which of our features are numerical and which are categorical.

In [6]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
Date             142193 non-null object
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null obje

Since we notice that 'Date' is an object instead of a datetime object, we convert it accordingly.

In [7]:
#set date to datetime object
rain['Date'] = pd.to_datetime(rain['Date'])
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Our data is all over Australia, which means there are lots of locations. Let's take all of these unique locations and store it on the variable 'locations'

In [8]:
#get unique locations
locations = rain['Location'].unique()
locations

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

Notice that 'Evaporation' and 'Sunshine' has ~40% missing values? To be honest we don't even know how this was measured so let's just drop these columns.

In [9]:
rain.drop(['Evaporation', 'Sunshine'], axis=1, inplace=True)

We know that Australia has 4 seasons. And we can't just fill the missing values with the mean value for each column. For example, 'MinTemp' or the Minimum Temperature of an area is changing depending on the season--what might be a 24 minimum temperature in Summer might be 14 in Winter. In order to be able to compute accurately, we must note what season it is for the missing value. The easiest way to denote the season is to see the month.

So let's create a new feature called 'Month'.

In [10]:
month = [d.month for d in rain['Date']]
rain['Month'] = month

And let's double check to see all our changes so far..

RISK_MM, Evaporation, and Sunshine columns should not be there anymore. And Month column should be there.

In [11]:
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,12
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,12
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,12
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,12
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,12


Now let's define a function to compute the seasonal numerical values. This takes a dataframe (which should already be minimised to its location--more of this later) and the column name of what we will be working with.

Separate the dataframe into seasons (spring, summer, fall, winter dataframes) and compute the mean value of whatever column we're working on.

Note that since our dataframe has missing WindGustSpeed and Pressure values even on location level, we must decide for default values on these features. If the computed mean value is NaN, then we use these default values.

In [12]:
def compute_missing_seasonal_num_values(dataframe, column_name):
    #separate to seasons
    defaults = {'WindGustSpeed': 5.4, 'Pressure9am': 1013.00, 'Pressure3pm': 1013.00}
    
    spring = dataframe[(dataframe['Month']) >= 9 & (dataframe['Month'] < 12)]
    sp_mean = np.mean(spring.loc[:,column_name])
    if (math.isnan(sp_mean) == True) | (np.isnan(sp_mean) == True):
        sp_mean = defaults[column_name]
    
    summer = dataframe[((dataframe['Month'] >= 1) | (dataframe['Month'] < 3)) & (dataframe['Month'] == 12)]
    sm_mean = np.mean(summer.loc[:,column_name])
    if (math.isnan(sm_mean) == True) | (np.isnan(sm_mean) == True):
        sm_mean = defaults[column_name]
    
    fall = dataframe[(dataframe['Month'] >= 3) & (dataframe['Month'] < 6)]
    fa_mean = np.mean(fall.loc[:,column_name])
    if (math.isnan(fa_mean) == True) | (np.isnan(fa_mean) == True):
        fa_mean = defaults[column_name]

    winter = dataframe[(dataframe['Month'] >= 6) & (dataframe['Month'] < 9)]
    wt_mean = np.mean(winter.loc[:,column_name])
    if (math.isnan(wt_mean) == True) | (np.isnan(wt_mean) == True):
        wt_mean = defaults[column_name]

    return sp_mean, sm_mean, fa_mean, wt_mean

I defined another function for filling the missing values on the dataframe itself. The function above only computes the values for each season.

In [13]:
def fill_missing_seasonal_num_values(dataframe, location, column_name):
    dfs = []
        
    sp, sm, fa, wt = compute_missing_seasonal_num_values(dataframe[dataframe['Location'] == location], column_name)
    df = dataframe[dataframe['Location'] == location]

    sp_df = df[(df['Month'] >= 9) & (df['Month'] < 12)]
    sp_df[column_name].fillna(sp, inplace=True)
    
    sm_df = df[((df['Month'] >= 1) & (df['Month'] < 3)) | (df['Month'] == 12)]
    sm_df[column_name].fillna(sm, inplace=True)
    
    fa_df = df[(df['Month'] >= 3) & (df['Month'] < 6)]
    fa_df[column_name].fillna(fa, inplace=True)
    
    wt_df = df[(df['Month'] >= 6) & (df['Month'] < 9)]
    wt_df[column_name].fillna(wt, inplace=True)

    dfs.append(sm_df)
    dfs.append(fa_df)
    dfs.append(wt_df)

    df = pd.concat(dfs)
        
    return df

Now let's fill up those missing numerical values.

In [14]:
cols = ['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm', 'Humidity9am', 'Humidity3pm',
        'Pressure9am', 'Pressure3pm', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']
df = rain.copy()
for col in cols:
    dfs=[]
    for location in locations:
        dfs.append(fill_missing_seasonal_num_values(df.copy(), location, col))
    df = pd.concat(dfs)

df.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,12
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,12
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,12
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,12
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,12


And let's check if it worked and set the main dataframe as the computed one.

In [15]:
df.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall          1080
WindGustDir       7001
WindGustSpeed        0
WindDir9am        8584
WindDir3pm        2892
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am         40532
Cloud3pm         43107
Temp9am              0
Temp3pm              0
RainToday         1080
RainTomorrow         0
Month                0
dtype: int64

In [16]:
rain = df
rain.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall          1080
WindGustDir       7001
WindGustSpeed        0
WindDir9am        8584
WindDir3pm        2892
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am         40532
Cloud3pm         43107
Temp9am              0
Temp3pm              0
RainToday         1080
RainTomorrow         0
Month                0
dtype: int64

'Rainfall' is the measurement of the amount of rainfall that occured on the corresponding 'Date'. We can assume that 0 means it didn't rain at all and has 0 mm precipitation. Let's fill it then with zeroes.

In [17]:
rain['Rainfall'].fillna(0, inplace=True)
rain.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir       7001
WindGustSpeed        0
WindDir9am        8584
WindDir3pm        2892
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am         40532
Cloud3pm         43107
Temp9am              0
Temp3pm              0
RainToday         1080
RainTomorrow         0
Month                0
dtype: int64

'RainToday' is filled up with the mode of the column.

In [18]:
rt_mode = rain['RainToday'].describe().top
rain['RainToday'].fillna(rt_mode, inplace=True)
rain.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir       7001
WindGustSpeed        0
WindDir9am        8584
WindDir3pm        2892
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am         40532
Cloud3pm         43107
Temp9am              0
Temp3pm              0
RainToday            0
RainTomorrow         0
Month                0
dtype: int64

'Cloud9am' and 'Cloud3pm' is measured using oktas. From 0 (clear sky) to 8 (completely overcast). We can fill the missing values with 0 okta which means it's a clear day.

In [19]:
rain['Cloud9am'].fillna(0, inplace=True)
rain['Cloud3pm'].fillna(0, inplace=True)

Now let's take care of the missing categorical values. Same as above, we define 2 functions -- (1) to compute the missing categorical values; (2) fill the missing values. In computing, I'm using the mode. But again, we compute this based on location and season.

In [20]:
def compute_missing_seasonal_cat_values(dataframe, column_name):
    #separate to seasons
    
    spring = dataframe[(dataframe['Month']) >= 9 & (dataframe['Month'] < 12)]
    sp_mode = spring[column_name].describe().top
    if (type(sp_mode) == float):
        sp_mode = dataframe[column_name].describe().top
    
    summer = dataframe[((dataframe['Month'] >= 1) | (dataframe['Month'] < 3)) & (dataframe['Month'] == 12)]
    sm_mode = summer[column_name].describe().top
    if (type(sm_mode) == float):
        sm_mode = dataframe[column_name].describe().top

    fall = dataframe[(dataframe['Month'] >= 3) & (dataframe['Month'] < 6)]
    fa_mode = fall[column_name].describe().top
    if (type(fa_mode) == float):
        fa_mode = dataframe[column_name].describe().top

    winter = dataframe[(dataframe['Month'] >= 6) & (dataframe['Month'] < 9)]
    wt_mode = winter[column_name].describe().top
    if (type(wt_mode) == float):
        wt_mode = dataframe[column_name].describe().top
       
    return sp_mode, sm_mode, fa_mode, wt_mode

In [21]:
def fill_missing_seasonal_cat_values(dataframe, location, column_name):
    dfs = []
        
    sp, sm, fa, wt = compute_missing_seasonal_cat_values(dataframe[dataframe['Location'] == location], column_name)
    df = dataframe[dataframe['Location'] == location]

    sp_df = df[(df['Month'] >= 9) & (df['Month'] < 12)]
    sp_df[column_name].fillna(sp, inplace=True)
    
    sm_df = df[((df['Month'] >= 1) & (df['Month'] < 3)) | (df['Month'] == 12)]
    sm_df[column_name].fillna(sm, inplace=True)
    
    fa_df = df[(df['Month'] >= 3) & (df['Month'] < 6)]
    fa_df[column_name].fillna(fa, inplace=True)
    
    wt_df = df[(df['Month'] >= 6) & (df['Month'] < 9)]
    wt_df[column_name].fillna(wt, inplace=True)

    dfs.append(sp_df)
    dfs.append(sm_df)
    dfs.append(fa_df)
    dfs.append(wt_df)

    df = pd.concat(dfs)
        
    return df

In [22]:
cols = ['WindGustDir', 'WindDir9am', 'WindDir3pm']
df = rain.copy()
for col in cols:
    dfs=[]
    for location in locations:
        dfs.append(fill_missing_seasonal_cat_values(df.copy(), location, col))
    df = pd.concat(dfs)

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,...,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,No,No,12
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,...,25.0,1010.6,1007.8,0.0,0.0,17.2,24.3,No,No,12
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,...,30.0,1007.6,1008.7,0.0,2.0,21.0,23.2,No,No,12
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,...,16.0,1017.6,1012.8,0.0,0.0,18.1,26.5,No,No,12
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,12


Again same with the problem awhile ago, WindGustDir has missing values on location level. I just filled it with the mode value of the overall dataframe. Considering that we've already dealt with the missing values of other locations.

In [23]:
top = df['WindGustDir'].describe().top
df['WindGustDir'].fillna(top, inplace=True)
df.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
Month            0
dtype: int64

In [24]:
rain = df

And final check...

In [25]:
rain.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
Month            0
dtype: int64

So now we can save this cleaned dataframe as a new csv file.

In [26]:
rain.to_csv('cleaned_weatherAUS.csv')